In [3]:
# Eval 3input model
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
#%matplotlib inline
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

INPUT_CHANNEL = 3
MEAN = [114.578, 115.294, 108.353]
DATASET = 'douyu_7100'
n_cl = 2

# load net
DEPLOY = './experiment/exper_douyu2700_googlenet/deploy.prototxt'
CAFFEMODEL = './experiment/exper_douyu2700_googlenet/snapshot/douyu2700_train_without_bn_iter_130000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)

imglist = open('./data/'+DATASET+'/test.txt').readlines()
datanum = len(imglist)

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n,n)

hist = 0

for img in imglist:
    img = img.strip()
    
    # load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
    im = Image.open('./data/'+DATASET+'/Images/'+img+'.png')
    in_ = np.array(im, dtype=np.float32)
    in_ = in_[:,:,::-1]
    in_ -= np.array(MEAN)
    in_ = in_.transpose((2,0,1))
    
    # load label
    label = sio.loadmat('./data/'+DATASET+'/Labels/'+img+'.mat')['label']
    label[label!=0]==1
    
    if INPUT_CHANNEL==4:
        imshape = (368, 640, 4)
        in_batch = np.ndarray(shape=(1, imshape[2], imshape[1], imshape[0]), dtype=np.float32)
        in_batch[0,0:3,:,:] = in_.copy()
        
        # load 4-th channel
        im_pred = Image.open('./data/'+DATASET+'/pred_xijin/{}.png_pred.png'.format(img))
        in_ = np.array(im_pred, dtype=np.float32)
        in_ -= 128
        in_batch[0,3,:,:] = in_.copy()
        
    elif INPUT_CHANNEL==3:
        imshape = (368, 640, 3)
        in_batch = np.ndarray(shape=(1, imshape[2], imshape[1], imshape[0]), dtype=np.float32)
        in_batch[0,0:3,:,:] = in_.copy()
        
    # shape for input (data blob is N x C x H x W), set data
    net.blobs['data'].reshape(*in_batch.shape)
    net.blobs['data'].data[...] = in_batch

    # run net and take argmax for prediction
    net.forward()
    out = net.blobs['score'].data[0].argmax(axis=0)
    
    this_hist =  fast_hist(label.flatten(),out.flatten(),n_cl)
    this_hist = np.array(this_hist, dtype=np.float32)
    this_iu = np.diag(this_hist) / (this_hist.sum(1) + this_hist.sum(0) - np.diag(this_hist))
    if np.isnan(this_iu[0]):
        this_iu[0] = 1
    if np.isnan(this_iu[1]):
        this_iu[1] = 1
    print img, this_iu, np.mean(this_iu) # first is neg iou, second is pos iou 
    hist += this_hist
    
hist = np.array(hist, dtype=np.float32)
iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
print '================================= result ================================='
print 'INPUT_CHANNEL: ',INPUT_CHANNEL
print 'MEAN', MEAN
print 'DEPLOY: ',DEPLOY
print 'CAFFEMODEL: ',CAFFEMODEL
print 'TEST DATASET: ',DATASET,
print 'TEST DATASET NUM: ',datanum
print 'IOU: ', iu, np.mean(iu)    




660209_1489763036_0 [ 0.85012418  0.98437703] 0.917251
douyuTV_660209_1488641433_5 [ 0.77723062  0.95767367] 0.867452
douyuTV_769170_1488942330_5 [ 0.80895084  0.97104245] 0.889997
douyuTV_1551804_1488811840_15 [ 0.88941169  0.93306333] 0.911237
douyuTV_579386_1488532523_0 [ 0.43142393  0.89561713] 0.663521
douyuTV_586697_1488550043_15 [ 0.76867747  0.94376612] 0.856222
douyuTV_825993_1489069177_0 [ 0.82721192  0.54473233] 0.685972
douyuTV_617410_1488552926_0 [ 0.94638902  0.97939909] 0.962894
douyuTV_1255190_1488457622_15 [ 0.96375692  0.99070698] 0.977232
douyuTV_586697_1489049502_0 [ 0.97622567  0.98969847] 0.982962
786909_1489894778_0 [ 0.51070631  0.9376905 ] 0.724198
438573_1489888999_0 [ 0.63836306  0.85796344] 0.748163
douyuTV_1551804_1488458252_10 [ 0.93407983  0.97713029] 0.955605
976326_1489783715_0 [ 0.90580016  0.97240013] 0.9391
684244_1489851864_0 [ 0.64010274  0.81811488] 0.729109
1255190_1489847960_0 [ 0.62418759  0.85280973] 0.738499
douyuTV_1546683_1488875586_0 [ 0.9